# **MML para el estudio de un Conjunto de Datos de Compras de Clientes**

**Contexto:**

¡Bienvenidos al mundo de las compras en Estambul! Nuestro conjunto de datos contiene información sobre compras realizadas en 10 centros comerciales diferentes entre 2021 y 2023. Hemos recopilado datos de diversos grupos de edad y géneros para ofrecer una visión integral de los hábitos de compra en Estambul. El conjunto de datos incluye información esencial como números de factura, IDs de clientes, edad, género, métodos de pago, categorías de productos, cantidad, precio, fechas de pedido y ubicaciones de los centros comerciales. Esperamos que este conjunto de datos sea un recurso valioso para investigadores, analistas de datos y entusiastas del aprendizaje automático que deseen obtener información sobre las tendencias y patrones de compra en Estambul. ¡Explora el conjunto de datos y descubre el fascinante mundo de las compras en Estambul!

**Contenido:**

**Información de los atributos:**

- **invoice_no:** Número de factura. Nominal. Una combinación de la letra 'I' y un entero de 6 dígitos asignado de manera única a cada operación.
- **customer_id:** Número de cliente. Nominal. Una combinación de la letra 'C' y un entero de 6 dígitos asignado de manera única a cada operación.
- **gender:** Variable de tipo cadena que indica el género del cliente.
- **age:** Variable de tipo entero positivo que indica la edad del cliente.
- **category:** Variable de tipo cadena que indica la categoría del producto comprado.
- **quantity:** Cantidad de cada producto (artículo) por transacción. Numérico.
- **price:** Precio unitario. Numérico. Precio del producto por unidad en Liras Turcas (TL).
- **payment_method:** Variable de tipo cadena que indica el método de pago (efectivo, tarjeta de crédito o tarjeta de débito) utilizado para la transacción.
- **invoice_date:** Fecha de la factura. El día en que se generó la transacción.
- **shopping_mall:** Variable de tipo cadena que indica el nombre del centro comercial donde se realizó la transacción.

In [39]:
import pandas as pd 
import numpy as np

In [40]:
url='/home/julio/jupyter_files/MLM_CustomerShoppingDataset/customer_shopping_data.csv'
df=pd.read_csv(url)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_no      99457 non-null  object 
 1   customer_id     99457 non-null  object 
 2   gender          99457 non-null  object 
 3   age             99457 non-null  int64  
 4   category        99457 non-null  object 
 5   quantity        99457 non-null  int64  
 6   price           99457 non-null  float64
 7   payment_method  99457 non-null  object 
 8   invoice_date    99457 non-null  object 
 9   shopping_mall   99457 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 7.6+ MB


**1. Preparación del Entorno**

In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

**2.Preprocesamiento**

In [43]:
# Variable objetivo y predictores
X = df.drop(columns=['price'])  # La variable objetivo es 'price'
y = df['price']

In [44]:
# Definición de las columnas numéricas y categóricas
numeric_features = ['age', 'quantity']
categorical_features = ['gender', 'category', 'payment_method', 'shopping_mall']

In [45]:
# Pipelines de preprocesamiento
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

In [46]:
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [47]:
# Combinación de preprocesamientos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

**3.Separación de los Datos en Conjuntos de Entrenamiento y Prueba**

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**4.Modelos de Regresión y Optimización**

**4.1 Regresión Lineal**

**Definición:**

La regresión lineal busca modelar la relación entre una variable dependiente $y$ y una o más variables independientes $X$. El modelo lineal tiene la forma:

$y = X \beta + \epsilon$

donde:
- $y$ es el vector de valores dependientes (respuestas),
- $X$ es la matriz de características,
- $\beta$ es el vector de coeficientes,
- $\epsilon$ es el vector de errores aleatorios.

**Objetivo:**

El objetivo es minimizar el error cuadrático medio (MSE):

$\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 $

donde $\hat{y}_i$ son las predicciones del modelo y $n$ es el número de observaciones.

**Coste:**

El coste para la regresión lineal es el MSE. En la regresión lineal, el modelo se ajusta minimizando la función de coste que es simplemente el MSE. Esto se puede expresar como:

$ \text{Coste}_{\text{Lineal}} = \frac{1}{n} \sum_{i=1}^{n} (y_i - (X_i \beta))^2$

donde $X_i \beta$ es la predicción del modelo para la $i$-ésima observación. El ajuste del modelo implica encontrar el vector de coeficientes $\beta$ que minimiza esta función de coste.

In [49]:
linear_model = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', LinearRegression())])

linear_model.fit(X_train, y_train)
y_pred_linear = linear_model.predict(X_test)

In [50]:
# Métricas de Evaluación
mse_linear = mean_squared_error(y_test, y_pred_linear)
r2_linear = r2_score(y_test, y_pred_linear)

print(f"Linear Regression - MSE: {mse_linear}, R2: {r2_linear}")

Linear Regression - MSE: 139575.27489624, R2: 0.8452172060098851


**4.2 Lasso (Optimizado para Ahorro de Memoria)**

**Definición:**

La regresión Lasso, o regularización L1, agrega una penalización a la suma de los valores absolutos de los coeficientes:

$\text{Coste}_{\text{Lasso}} = \| X \beta - y \|^2 + \alpha \sum_{j=1}^{p} |\beta_j| $

donde:
- $\sum_{j=1}^{p} |\beta_j| $ es la norma L1 de los coeficientes,
- $ \alpha $ es el parámetro de regularización.

**Efecto de la Penalización L1:**

- Penaliza la suma de los valores absolutos de los coeficientes.
- La penalización L1 tiende a forzar algunos coeficientes a ser exactamente cero, lo que lleva a una selección de características automática. Esto es útil para obtener modelos más simples y esqueléticos, eliminando características irrelevantes.

In [51]:
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Definir el modelo de Lasso sin el parámetro solver
lasso_model = Pipeline(steps=[('preprocessor', preprocessor),
                              ('regressor', Lasso(alpha=0.1, max_iter=500, tol=0.01, warm_start=True))])

# Ajustar el modelo
lasso_model.fit(X_train, y_train)

# Hacer predicciones
y_pred_lasso = lasso_model.predict(X_test)

/home/julio/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:639: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1485737936.8401604, tolerance: 701630337.9151617
  model = cd_fast.sparse_enet_coordinate_descent(


In [52]:
# Evaluar el modelo
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)

print(f"Lasso Regression - MSE: {mse_lasso}, R2: {r2_lasso}")


Lasso Regression - MSE: 139583.2730513902, R2: 0.8452083364102957


In [53]:
# Optimización del modelo de Lasso con GridSearchCV
param_grid_lasso = {
    'regressor__alpha': [0.01, 0.1, 1],
    'regressor__max_iter': [500,1000],
    'regressor__tol': [0.001,0.01]
}

grid_search_lasso = GridSearchCV(lasso_model, param_grid_lasso, cv=3, scoring='neg_mean_squared_error')
grid_search_lasso.fit(X_train, y_train)

/home/julio/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:639: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 965796899.7268643, tolerance: 46651017.265565336
  model = cd_fast.sparse_enet_coordinate_descent(
/home/julio/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:639: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 967063348.7882781, tolerance: 46538358.34233699
  model = cd_fast.sparse_enet_coordinate_descent(
/home/julio/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:639: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 990834741.4657626, tolerance: 47133549.332827955
  model = cd_fast.sparse_enet_coordinate_descent(
/home/julio/.local/lib/python3.10/site-packages/sklearn/linear_model/_c

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['age',
                                                                          'quantity']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['gender',
                                                                          'category',
                                                                          'payment_method',
                                                                          'shopping_mall'])])),
                                       ('regressor',
                                        Lasso(alpha=0.1, max_iter=500, tol=0.01,
                                              warm_start=True))]),
             param_grid={'regressor__alpha': [0.01, 0.1, 1],
                         'regressor__max_iter': [500, 1000],
                         'regressor__tol': [0.001, 0.01]},
             scoring='neg_mean_squared_error')

In [55]:
# Mejor modelo de Lasso
best_lasso_model = grid_search_lasso.best_estimator_

# Predicciones del mejor modelo de Lasso
y_pred_best_lasso = best_lasso_model.predict(X_test)

# Métricas de Evaluación del mejor modelo de Lasso
mse_best_lasso = mean_squared_error(y_test, y_pred_best_lasso)
r2_best_lasso = r2_score(y_test, y_pred_best_lasso)

print(f"Optimized Lasso Regression - MSE: {mse_best_lasso}, R2: {r2_best_lasso}")

Optimized Lasso Regression - MSE: 139583.27305139063, R2: 0.8452083364102952


**4.3 Ridge**

**Definición:**

La regresión Ridge, o regularización L2, agrega una penalización a la suma de los cuadrados de los coeficientes:

$\text{Coste}_{\text{Ridge}} = \| X \beta - y \|^2 + \alpha \| \beta \|_2^2 $

donde:
- $ \| \beta \|_2^2 = \sum_{j=1}^{p} \beta_j^2 $ es la norma L2 de los coeficientes,
- $ \alpha $ es el parámetro de regularización.

**Efecto de la Penalización L2:**

- Penaliza la magnitud total de los coeficientes.
- La penalización L2 tiende a reducir el tamaño de todos los coeficientes, pero no necesariamente los lleva a cero. Esto es útil para manejar la multicolinealidad y evitar el sobreajuste al reducir la magnitud de los coeficientes grandes.

In [56]:
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Definir el modelo Ridge
ridge_model = Pipeline(steps=[('preprocessor', preprocessor),
                              ('regressor', Ridge(alpha=1.0, solver='auto'))])

# Ajustar el modelo
ridge_model.fit(X_train, y_train)

# Hacer predicciones
y_pred_ridge = ridge_model.predict(X_test)

In [57]:
# Evaluar el modelo
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

print(f"Ridge Regression - MSE: {mse_ridge}, R2: {r2_ridge}")

Ridge Regression - MSE: 139581.7558460975, R2: 0.8452100189237239


In [58]:
# Optimización del modelo Ridge con GridSearchCV
param_grid_ridge = {
    'regressor__alpha': [0.01, 0.1, 1, 10],
    'regressor__solver': ['auto', 'lsqr', 'sparse_cg', 'sag', 'lbfgs']
}

grid_search_ridge = GridSearchCV(ridge_model, param_grid_ridge, cv=5, scoring='neg_mean_squared_error')
grid_search_ridge.fit(X_train, y_train)

# Mejor modelo Ridge
best_ridge_model = grid_search_ridge.best_estimator_

# Predicciones del mejor modelo Ridge
y_pred_best_ridge = best_ridge_model.predict(X_test)

/home/julio/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
20 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/home/julio/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/julio/.local/lib/python3.10/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/home/julio/.local/lib/python3.10/site-packages/sklearn/pipeline.py", line 475, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["f

In [59]:
# Métricas de Evaluación del mejor modelo Ridge
mse_best_ridge = mean_squared_error(y_test, y_pred_best_ridge)
r2_best_ridge = r2_score(y_test, y_pred_best_ridge)

print(f"Optimized Ridge Regression - MSE: {mse_best_ridge}, R2: {r2_best_ridge}")

Optimized Ridge Regression - MSE: 139581.7558460975, R2: 0.8452100189237239


**4.4 Elastic Net**

**Definición:**

La regresión Elastic Net combina las penalizaciones L1 y L2:

$\text{Coste}_{\text{ElasticNet}} = \| X \beta - y \|^2 + \alpha (\rho \| \beta \|_1 + \frac{1 - \rho}{2} \| \beta \|_2^2)$

donde:
- $\rho$ es un parámetro que balancea entre L1 y L2,
- $\alpha$ es el parámetro de regularización.

**Efecto de la Penalización Elastic Net:**

- Combina las ventajas de L1 y L2. Proporciona una forma de selección de características (como Lasso) y manejo de la multicolinealidad (como Ridge).
- Es útil cuando hay muchas características correlacionadas o cuando se desea combinar la selección de características con la reducción de magnitud de coeficientes.

In [60]:
elastic_net_model = Pipeline(steps=[('preprocessor', preprocessor),
                                    ('regressor', ElasticNet(alpha=0.1, l1_ratio=0.5))])

elastic_net_model.fit(X_train, y_train)
y_pred_elastic_net = elastic_net_model.predict(X_test)

In [61]:
# Métricas de Evaluación
mse_elastic_net = mean_squared_error(y_test, y_pred_elastic_net)
r2_elastic_net = r2_score(y_test, y_pred_elastic_net)

print(f"Elastic Net Regression - MSE: {mse_elastic_net}, R2: {r2_elastic_net}")

Elastic Net Regression - MSE: 261495.2337151292, R2: 0.7100133750793987


In [62]:
# Optimización del modelo Elastic Net
param_grid_elastic_net = {
    'regressor__alpha': [0.01, 0.1, 1.0, 5.0],
    'regressor__l1_ratio': [0.1, 0.5, 0.9],
    'regressor__max_iter': [500, 1000]
}

grid_search_elastic_net = GridSearchCV(elastic_net_model, param_grid_elastic_net, cv=5, scoring='neg_mean_squared_error')
grid_search_elastic_net.fit(X_train, y_train)

# Mejor modelo Elastic Net
best_elastic_net_model = grid_search_elastic_net.best_estimator_
y_pred_best_elastic_net = best_elastic_net_model.predict(X_test)

In [63]:
# Métricas de Evaluación del mejor modelo Elastic Net
mse_best_elastic_net = mean_squared_error(y_test, y_pred_best_elastic_net)
r2_best_elastic_net = r2_score(y_test, y_pred_best_elastic_net)

print(f"Optimized Elastic Net Regression - MSE: {mse_best_elastic_net}, R2: {r2_best_elastic_net}")

Optimized Elastic Net Regression - MSE: 140207.4532627178, R2: 0.8445161482191247


**4.5 Relación entre MSE y Coste**

**MSE (Error Cuadrático Medio):**

$\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$

Es la medida de la diferencia promedio al cuadrado entre las predicciones del modelo y los valores reales. Es fundamental para evaluar el ajuste del modelo.

**Coste en Regularización:**

En los modelos de regresión con regularización (Ridge, Lasso, Elastic Net), el coste es una combinación del MSE y la penalización por la magnitud de los coeficientes:

- **Ridge:** $\text{Coste}_{\text{Ridge}} = \text{MSE} + \alpha \| \beta \|_2^2 $
- **Lasso:** $\text{Coste}_{\text{Lasso}} = \text{MSE} + \alpha \sum_{j=1}^{p} |\beta_j| $
- **Elastic Net:** $\text{Coste}_{\text{ElasticNet}} = \text{MSE} + \alpha (\rho \| \beta \|_1 + \frac{1 - \rho}{2} \| \beta \|_2^2)$

La penalización ajusta el MSE para evitar el sobreajuste y mejorar la generalización del modelo al castigar la complejidad del modelo. La elección del tipo y magnitud de la penalización afecta la interpretación del modelo y la selección de características.

**4.6. Residuos (Errores de Predicción)**

- **Definición:** Los residuos son las diferencias entre los valores reales y los valores predichos por el modelo de regresión. Para cada observación, el residuo se calcula como:

  $
  e_i = y_i - \hat{y}_i
 $

  Donde:
  -$y_i$ es el valor real de la $i$-ésima observación.
  - $\hat{y}_i$ es el valor predicho por el modelo para la $i$-ésima observación.
  - $e_i$ es el residuo o error para la $i$-ésima observación.

- **Interpretación:** Los residuos indican cuánto se desvían las predicciones del modelo de los valores reales. Un residuo grande sugiere que el modelo está haciendo una predicción inexacta para esa observación en particular.

- **Uso:** En el cálculo de R², los residuos se utilizan para medir la discrepancia entre las predicciones del modelo y los datos reales. La suma de los cuadrados de los residuos $\text{SS}_{\text{res}}$ es una medida de cuánto error comete el modelo.

**4.7 Suma Total de Cuadrados (SS_tot)**

- **Definición:** La suma total de cuadrados es una medida de la variabilidad total en los datos. Se calcula como:

 $
  \text{SS}_{\text{tot}} = \sum_{i=1}^{n} (y_i - \bar{y})^2
$

  Donde:
  - $y_i$ es el valor real de la $i$-ésima observación.
  - $\bar{y}$ es la media de los valores reales $ \bar{y} = \frac{1}{n} \sum_{i=1}^{n} y_i $.
  - $\text{SS}_{\text{tot}}$ es la suma total de cuadrados.

- **Interpretación:** La suma total de cuadrados representa la variabilidad total de los datos alrededor de su media. Es una medida de cuánta variación hay en los datos en ausencia de un modelo.

- **Uso:** En el cálculo de R², $\text{SS}_{\text{tot}}$ se usa como el denominador para medir la proporción de variabilidad explicada por el modelo. 

### **Cálculo del Coeficiente de Determinación (R²)**

El coeficiente de determinación $R^2$ se calcula usando la siguiente fórmula:

$
R^2 = 1 - \frac{\text{SS}_{\text{res}}}{\text{SS}_{\text{tot}}}
$

Donde:
- $\text{SS}_{\text{res}}$ es la suma de los cuadrados de los residuos, que se calcula como:

  $
  \text{SS}_{\text{res}} = \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
  $

- $\text{SS}_{\text{tot}}$ es la suma total de cuadrados.

### **Interpretación de R²**

- **R² = 1:** El modelo explica toda la variabilidad en los datos. Los residuos son cero, y el modelo predice perfectamente todos los valores reales.
- **R² = 0:** El modelo no explica ninguna variabilidad adicional en los datos más allá de la media. En este caso, el modelo es tan bueno como simplemente predecir la media de los valores.
- **R² < 0:** El modelo es peor que simplemente predecir la media de los valores, lo que sugiere que el modelo está haciendo un mal ajuste.

### **Conclusión**

- **Residuos** miden la discrepancia entre las predicciones del modelo y los valores reales.
- **Suma Total de Cuadrados (SS_tot)** mide la variabilidad total en los datos respecto a la media de los datos.
- **R²** proporciona una proporción de la variabilidad total que es explicada por el modelo, comparando la variabilidad explicada por el modelo con la variabilidad total en los datos.

In [66]:
# Crear el DataFrame
data = {
    'Modelo': [
        'Linear Regression', 
        'Lasso Regression', 
        'Optimized Lasso Regression',
        'Ridge Regression', 
        'Optimized Ridge Regression', 
        'Elastic Net Regression', 
        'Optimized Elastic Net Regression'
    ],
    'MSE': [
        139575.27489624, 
        139583.2730513902, 
        139583.27305139063, 
        139581.7558460975, 
        139581.7558460975, 
        261495.2337151292, 
        140207.4532627178
    ],
    'R2': [
        0.8452172060098851, 
        0.8452083364102957, 
        0.8452083364102952, 
        0.8452100189237239, 
        0.8452100189237239, 
        0.7100133750793987, 
        0.8445161482191247
    ]
}

In [67]:
df1 = pd.DataFrame(data)

print(df1)

                             Modelo            MSE        R2
0                 Linear Regression  139575.274896  0.845217
1                  Lasso Regression  139583.273051  0.845208
2        Optimized Lasso Regression  139583.273051  0.845208
3                  Ridge Regression  139581.755846  0.845210
4        Optimized Ridge Regression  139581.755846  0.845210
5            Elastic Net Regression  261495.233715  0.710013
6  Optimized Elastic Net Regression  140207.453263  0.844516


**Interpretación y Comparación**

**1. Linear Regression:**
MSE: 139575.27
R2: 0.8452
La regresión lineal proporciona una buena predicción con un R2 bastante alto, indicando que el modelo explica aproximadamente el 84.52% de la variabilidad en los datos.

**2. Lasso Regression:**
MSE: 139583.27
R2: 0.8452
La regresión Lasso muestra un rendimiento muy similar al de la regresión lineal, con un MSE ligeramente mayor y un R2 casi idéntico. La regularización L1 no parece haber afectado significativamente el rendimiento en este caso.

**3. Optimized Lasso Regression:**
MSE: 139583.27
R2: 0.8452
El rendimiento de la regresión Lasso optimizada es prácticamente igual al de la regresión Lasso básica. Esto sugiere que la optimización no ha producido mejoras significativas en el MSE o en el R2.

**4. Ridge Regression:**
MSE: 139581.76
R2: 0.8452
La regresión Ridge también muestra un MSE y R2 muy similares a los de la regresión lineal. La regularización L2 no ha cambiado notablemente el ajuste del modelo, similar al caso de Lasso.

**5. Optimized Ridge Regression:**
MSE: 139581.76
R2: 0.8452
La regresión Ridge optimizada muestra resultados idénticos a los de la regresión Ridge básica, indicando que la optimización no ha producido mejoras adicionales en este caso.

**6. Elastic Net Regression:**
MSE: 261495.23
R2: 0.7100
La regresión Elastic Net presenta un MSE significativamente mayor y un R2 mucho más bajo en comparación con los modelos anteriores. Esto sugiere que Elastic Net no se está ajustando bien a los datos, posiblemente debido a la combinación de penalizaciones L1 y L2, que puede no ser adecuada para este conjunto de datos en particular.

**7. Optimized Elastic Net Regression:**
MSE: 140207.45
R2: 0.8445
La versión optimizada de Elastic Net mejora el MSE y el R2 en comparación con la versión básica de Elastic Net, pero sigue siendo inferior a los modelos de regresión lineal, Lasso y Ridge. La optimización no logra igualar el rendimiento de los otros modelos.

**Conclusión:**

*Modelos Lineales (Regresión Lineal, Lasso y Ridge):* Los modelos lineales muestran un rendimiento similar, con valores de MSE y R2 muy cercanos entre sí. La regularización L1 (Lasso) y L2 (Ridge) no mejoran significativamente el ajuste en comparación con la regresión lineal básica.

*Elastic Net:* Elastic Net muestra un rendimiento inferior en comparación con los modelos lineales, tanto en términos de MSE como de R2. La optimización del Elastic Net mejora algo el rendimiento, pero aún no alcanza a los modelos lineales básicos.

En general, para este conjunto de datos, los modelos lineales proporcionan los mejores resultados en términos de precisión de predicción, mientras que la elastic net, incluso optimizada, no logra un rendimiento competitivo.

In [68]:
import matplotlib.pyplot as plt
import os
# Obtén la ruta del archivo actual
current_directory = os.getcwd()

# Configura el tamaño de la figura y el estilo
fig, ax = plt.subplots(figsize=(10, 3))  # Puedes ajustar el tamaño según sea necesario
ax.axis('off')

# Crea la tabla
table = ax.table(cellText=df1.values,
                 colLabels=df1.columns,
                 cellLoc='center',
                 loc='center',
                 colColours=['#f5f5f5']*len(df1.columns),
                 bbox=[0, 0, 1, 1])

# Ajusta el diseño
plt.tight_layout()

# Define el nombre del archivo de salida
output_file = os.path.join(current_directory, 'dataframe_image.png')

# Guarda la imagen
plt.savefig(output_file, bbox_inches='tight', pad_inches=0.1)
plt.close()  # Cierra la figura para liberar memoria

print(f'Imagen guardada en: {output_file}')

Imagen guardada en: /home/julio/jupyter_files/MLM_CustomerShoppingDataset/dataframe_image.png


🔍 Modelos de Machine Learning-Modelos de Regresión: Evaluación de MSE y R²

Estoy emocionado de compartir los resultados de mi reciente análisis de modelos de regresión. En este análisis, evalué distintas técnicas de regresión, tanto estándar como optimizadas, para entender mejor su rendimiento y precisión.

🔹 MSE (Error Cuadrático Medio): Mide la precisión de las predicciones del modelo; valores más bajos indican mejor rendimiento.

🔹 R² (Coeficiente de Determinación): Representa la proporción de la variabilidad en la variable dependiente que es explicada por el modelo. Un valor más cercano a 1 indica un modelo que explica bien los datos.

Observaciones:

Los modelos de Regresión Lineal, Lasso y Ridge, tanto en su versión estándar como optimizada, presentan un rendimiento muy similar.
La Regresión Elastic Net muestra un MSE más alto y un R² más bajo en comparación con los otros modelos, sugiriendo que podría no ser la mejor opción para este conjunto de datos específico.

La optimización de Lasso y Ridge no muestra mejoras significativas en comparación con sus versiones estándar, lo que indica que el ajuste de hiperparámetros puede no haber tenido un impacto relevante en este caso.

#DataScience #MachineLearning #Regresión #AnálisisDeDatos #ModelosPredictivos